首先是如何使用单个GPU，然后是如何使用多个GPU和多个服务器（具有多个GPU）。

In [1]:
!nvidia-smi

Mon Jan  9 21:14:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.00       Driver Version: 517.00       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P0    33W /  N/A |      0MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

计算设备
默认情况下，张量是在内存中创建的，然后使用CPU计算它

在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。
应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。 然而，gpu设备只代表一个卡和相应的显存。 如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第块GPU（从0开始）。 另外，cuda:0和cuda是等价的。

In [2]:
import torch
from torch import nn

In [3]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

我们可以查询gpu的数量

In [4]:
torch.cuda.device_count()

1

现在我们定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [5]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

张量与GPU
我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。
例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设我们至少有两个gpu，下面的代码将在第二个GPU上创建一个随机张量。

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y, Y.device

(tensor([[0.2055, 0.0852, 0.0090],
         [0.6560, 0.0833, 0.1265]]),
 device(type='cpu'))

如果我们要计算X + Y，我们需要决定在哪里执行这个操作。 例如，如 图5.6.1所示， 我们可以将X传输到第二个GPU并在那里执行操作。 不要简单地X加上Y，因为这会导致异常， 运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。 由于Y位于第二个GPU上，所以我们需要将X移到那里， 然后才能执行相加运算。

In [9]:
# Z = X.cuda(1)
# print(X)
# print(Z)

In [10]:
# X + Y

神经网络与GPU

类似地，神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上。

In [11]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

当输入为GPU上的张量时，模型将在同一GPU上计算结果。

In [12]:
net(X)

tensor([[1.2306],
        [1.2306]], device='cuda:0', grad_fn=<AddmmBackward0>)

让我们确认模型参数存储在同一个GPU上。

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

总之，只要所有的数据和参数都在同一个设备上， 我们就可以有效地学习模型。 在下面的章节中，我们将看到几个这样的例子。

小结：
1 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。
2 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
3 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。